In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
import torchvision
import os
#import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import numpy as np
#import torch.utils.data.distributed
##import torchvision.models as models
#import torch.nn.parallel
#import torch.backends.cudnn as cudnn
#import torch.distributed as dist
from torch.optim.lr_scheduler import LambdaLR
#parameters
epochs=20
bs=128
lr=0.1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Create Model
def get_model():
 model=torchvision.models.resnet50(pretrained=False)
 return model,optim.SGD(model.parameters(),lr=lr)
model,opt=get_model()
model.half()
for layer in model.modules():
  if isinstance(layer,nn.BatchNorm2d):
    layer.float()
model.cuda()

#lambda
def x(epoch):
  if epoch <=5:
     return epoch/5
  else:
     return 1
lambda1 = lambda epoch: x(epoch)
scheduler = LambdaLR(opt, [lambda1])



#Loss function
loss_func=F.cross_entropy
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)
    print(loss)
    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

   
#DATA LOADER  

train_dir=os.path.join('/content/gdrive/My Drive/imagenette2/','train')
val_dir=os.path.join('/content/gdrive/My Drive/imagenette2/','val')       
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
train_dataset = datasets.ImageFolder(train_dir,transforms.Compose([transforms.RandomResizedCrop(224),
transforms.RandomHorizontalFlip(),transforms.ToTensor(),normalize]))
train_loader=DataLoader(train_dataset,batch_size=bs,shuffle=True,pin_memory=True)
val_loader=DataLoader(datasets.ImageFolder(val_dir,transforms.Compose([transforms.Resize(256),
transforms.CenterCrop(224),transforms.ToTensor(),normalize])),batch_size=bs,shuffle=False,pin_memory=True)


for epoch in range(epochs):
        model.train()
        for xb, yb in train_loader:
            xb=xb.to('cuda').half()
            yb=yb.to('cuda')
            loss_batch(model, loss_func, xb, yb, opt)

        model.eval()
        with torch.no_grad():
            
            losses, nums = zip(
                *[loss_batch(model, loss_func, xb.to('cuda').half(), yb.to('cuda')) for xb, yb in val_loader]
            )
        val_loss = np.sum(np.multiply(losses, nums)) / np.sum(nums)
        
        print(epoch, val_loss) 
        scheduler.step()
        print(scheduler.get_last_lr())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
tensor(7.1367, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.0742, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.0625, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.0703, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.0859, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.1914, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.0117, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.1211, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.2383, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.0742, device='cuda:0', dtype=torch.float16, grad_fn=<NllLossBackward>)
tensor(7.1953, device='cuda:0', dtype=torch.float16, grad_fn=<NllLoss

KeyboardInterrupt: ignored